Not sure how much help it will be but this notebook describes how I made a WAG about which patients are in the control group and which are Parkinson's patients.

Summary:

1.  I use that to fill missing values in the training updr_4.
2.  Some shared notebooks are dropping rows where medication status is unknown - I think that's a huge mistake.  Think I saw one that dropped missing value rows for updrs_4, also a mistake IMO.
3.  Both medication status and control/patient are pretty good features in a few models (not shown in this notebook) - but we don't get that information in test (meds) and it might be hard to figure out control using the method in this notebook.
4.  Might want to look at a way to use control group info with proteins/peptides - ?

In [ ]:
FILE_DIR = '/kaggle/input/amp-parkinsons-disease-progression-prediction/'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# increase the number of rows during debug as some of my logic was not easy for me to code - 
pd.options.display.max_rows = 50   # 500


In [ ]:
FILL_MEDS = 'UNKNOWN'
HOW = 'median'          

In [ ]:
def Build_Initial(fill_stuff):
    trn = pd.read_csv(FILE_DIR + 'train_clinical_data.csv')
    # trn = pd.read_csv('/kaggle/input/amp-pd/train_clinical_data.csv')
    targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
    # Fill the nan with a new state - UNKNOWN
    if fill_stuff == 'None':
        trn.upd23b_clinical_state_on_medication = trn.drop('upd23b_clinical_state_on_medication',axis=1).dropna()
    else:
        trn.upd23b_clinical_state_on_medication = trn.upd23b_clinical_state_on_medication.fillna(fill_stuff)
    return trn, targets

In [ ]:
trn, targets = Build_Initial(fill_stuff = FILL_MEDS)
trn

In [ ]:
# Group the data by 'upd23b_clinical_state_on_medication' and select the 'updrs_1' column
grouped = trn.groupby('upd23b_clinical_state_on_medication')['updrs_1']

# Create a figure with twice the default size
fig, ax = plt.subplots(figsize=(12, 12))
# Plot a histogram of 'updrs_1' for each group
grouped.hist(alpha=0.5, bins=100, density=True)
# Add a legend to the plot
ax.legend(title='Medication State', loc='upper right')
# Add titles and labels to the plot
plt.title('Histogram of UPDRS-1 by Medication State')
plt.xlabel('UPDRS-1 Score')
plt.ylabel('Density')

# Show the plot
plt.show()

Two distinct distributions of all UPDRS exist based on the medication status.  Dropping the nan med status rows would appear to be a mistake.  While it's likely Off meds, its more likely that it's meds never proscribed.


Used this Fill_survey in local models.  Median always better than mean.
After some experiments I ended up using median to fill missing for parts 1-3.

In [ ]:
def Fill_survey(trn, HOW):
    
    if HOW == 'median':
        print('using median')
        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_1'].median()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_1'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_1']) else x['updrs_1'], axis=1)

        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_2'].median()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_2'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_2']) else x['updrs_2'], axis=1)

        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_3'].median()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_3'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_3']) else x['updrs_3'], axis=1)

#         grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_4'].median()
#         # Fill missing values in 'updrs_1' with the mean of each corresponding group
#         trn['updrs_4'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_4']) else x['updrs_4'], axis=1)

        
    elif HOW == 'mean':
        print('using mean')
        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_1'].mean()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_1'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_1']) else x['updrs_1'], axis=1)

        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_2'].mean()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_2'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_2']) else x['updrs_2'], axis=1)

        grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_3'].mean()
        # Fill missing values in 'updrs_1' with the mean of each corresponding group
        trn['updrs_3'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_3']) else x['updrs_3'], axis=1)

#         grouped = trn.groupby(['visit_month', 'upd23b_clinical_state_on_medication'])['updrs_4'].mean()
#         # Fill missing values in 'updrs_1' with the mean of each corresponding group
#         trn['updrs_4'] = trn.apply(lambda x: grouped.loc[(x['visit_month'], x['upd23b_clinical_state_on_medication'])] if pd.isna(x['updrs_4']) else x['updrs_4'], axis=1)
        
        

    return trn

Changed the name to save on keystrokes :)

In [ ]:
trn = Fill_survey(trn, HOW)
trn = trn.rename(columns={'upd23b_clinical_state_on_medication': 'meds'})

In [ ]:
import matplotlib.ticker as mtick

(trn.isna().sum()/trn.shape[0]).plot(kind='barh', title = 'NA value counts')
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
plt.show()

In [ ]:
# Guess #1 - patients with medication status of Unknown are control.
trn['control'] = 0.0          # set all patients as Parkinson's    
trn.loc[trn['meds'].isin(['UNKNOWN', 'Off']), 'control'] = 1.0
trn['control'] = trn.groupby('patient_id')['control'].transform(lambda x: 1 if x.sum() == len(x) else 0)

trn.head(5)

In [ ]:
unique_patients = trn.groupby('control')['patient_id'].nunique()
print(unique_patients)

With this first attempt to determine if an patient is in the control group we have found 99 possible control group patients.


Guess #2 - control group patients will have only 0 or nan values for part 4.
When you take the survey yourself you will see that part 4 will have 0 score for a normal healthy person.  Hard to believe that a Parkison's patient would score 0 here, even when meds working great.  I am old guy with pretty high scores in parts 1-3 but 0.0 in part 4.

In [ ]:
# fill in the nan - I had a lot of trouble getting my logic to work with NaN - not sure why but this fixed an unknown issue
trn['updrs_4'].fillna(-1.0, inplace=True)
trn['control_2'] = 0.0

# Define a function to update the 'control_2' column based on the condition
def update_control_2(group):
    if group['updrs_4'].apply(lambda x: x == 0.0 or x == -1.0).all():
        group['control_2'] = 1.0
    return group

# Apply the update_control_2 function to each group of rows with the same 'patient_id'
trn = trn.groupby('patient_id').apply(update_control_2)


trn.head(5)

In [ ]:
unique_patients = trn.groupby('control_2')['patient_id'].nunique()
print(unique_patients)

Now a different possible split of control vs patients.  Like this one better as 50/50 seems like a nice balanced study.
Wanted to do another possible evaluation as a tie breaker but could not think of one.

So - lets do a simple assumption that control patients both would score 1.0 with my simple tests.

In [ ]:
# Set the data types of 'control' and 'control_2' columns to float32
trn['control'] = trn['control'].astype('float32')
trn['control_2'] = trn['control_2'].astype('float32')

trn['control_sum'] = trn['control'] + trn['control_2']
unique_patients = trn.groupby('control_sum')['patient_id'].nunique()
print(unique_patients)

We have 92 patients that we have some confidence are control group.
Lets set updrs_4 to value of 0.0 for our control group when -1.0
Lets also update the meds to None for our suspect control group

In [ ]:
# Update 'updrs_4' column based on the condition
trn.loc[(trn['control_sum'] == 2.0) & (trn['updrs_4'] == -1.0), 'updrs_4'] = 0.0

# Update 'meds' column based on the condition
trn.loc[(trn['control_sum'] == 2.0) & (trn['meds'] == 'UNKNOWN'), 'meds'] = 'None'

# Update 'meds' column based on the condition - these are most likely Parkinson's and going to assume Off
trn.loc[(trn['control'] == 0.0) & (trn['meds'] == 'UNKNOWN'), 'meds'] = 'Off'

# Update 'updrs_4' column based on the condition
trn.loc[trn['updrs_4'] == -1.0, 'updrs_4'] = 0.0


trn.head(5)

Any patient with control_sum of 2.0 is most likely a control subject.
Controls will not have Parkinson's but they might have/develop other issues where peptides play a role in scoring.
But in general I would assume they have a fairly flat score for parts 1-3 over time.
My personal scores are mid-range in those parts, likly the result of age (76) and that I have been pushing the diabetes border for a few years.

Still have a few that I am not sure of - but for now 4 groups of 'meds'.

We don't get meds status information in Test and still evaluating how I will use control info and meds


In [ ]:
print(trn['meds'].nunique())

In [ ]:
grouped = trn.groupby('control_sum')['updrs_1']

# Create a figure with twice the default size
fig, ax = plt.subplots(figsize=(12, 12))

# Define the histogram properties
num_bins = 100
bar_width = 0.8
offset = 0.25

# Define the groups and colors
groups = list(grouped.groups.keys())
colors = ['C0', 'C1', 'C2']

# Loop through the groups and plot the histogram with custom spacing
for i, (name, group) in enumerate(grouped):
    counts, bin_edges = np.histogram(group, bins=num_bins, density=True)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    ax.bar(bin_centers + (i - 1) * offset, counts, width=bar_width / len(groups), alpha=1, color=colors[i], label=name)

# Add a legend to the plot
ax.legend(title='Control vs Parkinsons State', loc='upper right')
# Add titles and labels to the plot
plt.title('Histogram of UPDRS-1 by Control State')
plt.xlabel('UPDRS-1 Score')
plt.ylabel('Density')

# Set y-axis limit
# ax.set_ylim(bottom=0, top=15)

# Show the plot
plt.show()


Kind of looks like I really should move move of the patient_id into the control grouping.

In [ ]:
grouped = trn.groupby('control_sum')['updrs_2']

# Create a figure with twice the default size
fig, ax = plt.subplots(figsize=(12, 12))

# Define the histogram properties
num_bins = 100
bar_width = 0.8
offset = 0.25

# Define the groups and colors
groups = list(grouped.groups.keys())
colors = ['C0', 'C1', 'C2']

# Loop through the groups and plot the histogram with custom spacing
for i, (name, group) in enumerate(grouped):
    counts, bin_edges = np.histogram(group, bins=num_bins, density=True)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    ax.bar(bin_centers + (i - 1) * offset, counts, width=bar_width / len(groups), alpha=1, color=colors[i], label=name)

# Add a legend to the plot
ax.legend(title='Control vs Parkinsons State', loc='upper right')
# Add titles and labels to the plot
plt.title('Histogram of UPDRS-2 by Control State')
plt.xlabel('UPDRS-2 Score')
plt.ylabel('Density')

# Set y-axis limit
# ax.set_ylim(bottom=0, top=15)

# Show the plot
plt.show()


In [ ]:
# Group the data by 'upd23b_clinical_state_on_medication' and select the 'updrs' column
grouped = trn.groupby('control_sum')['updrs_3']

# Create a figure with twice the default size
fig, ax = plt.subplots(figsize=(12, 12))
# Plot a histogram of 'updrs' for each group
grouped.hist(alpha=1, bins=100, density=True)  # Increase the alpha value to make the bars less transparent
# Add a legend to the plot
ax.legend(title='Control vs Parkinsons State', loc='upper right')
# Add titles and labels to the plot
plt.title('Histogram of UPDRS-3 by Control State')
plt.xlabel('UPDRS-3 Score')
plt.ylabel('Density')

# Set y-axis limit
# ax.set_ylim(bottom=0, top=4)  # Adjust the y-axis limit to see the group with all zero values better

# Show the plot
plt.show()


Ok - this plot makes me think I have the control group split right - how to explain the chart for part 1?

When you take the survey yourself I think it's understandable that control folks will score differently than Parkinson's patients.

In [ ]:
grouped = trn.groupby('control_sum')['updrs_4']

# Create a figure with twice the default size
fig, ax = plt.subplots(figsize=(12, 12))

# Define the histogram properties
num_bins = 100
bar_width = 0.8
offset = 0.25

# Define the groups and colors
groups = list(grouped.groups.keys())
colors = ['C0', 'C1', 'C2']

# Loop through the groups and plot the histogram with custom spacing
for i, (name, group) in enumerate(grouped):
    counts, bin_edges = np.histogram(group, bins=num_bins, density=True)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    ax.bar(bin_centers + (i - 1) * offset, counts, width=bar_width / len(groups), alpha=1, color=colors[i], label=name)

# Add a legend to the plot
ax.legend(title='Control vs Parkinsons State', loc='upper right')
# Add titles and labels to the plot
plt.title('Histogram of UPDRS-4 by Control State')
plt.xlabel('UPDRS-4 Score')
plt.ylabel('Density')

# Set y-axis limit
ax.set_ylim(bottom=0, top=15)

# Show the plot
plt.show()


While taking the survey my impression was that part 1 and 2 might not show a lot of differences between controls and Parkinson's patients.  Not sure of the age distribution for occurrence of Parkinson's, and would assume that any control group would be a reasonable age match.

Now what to do with this knowledge?

In [ ]:
nan